In [7]:
pip install -qU "langchain[google-genai]"

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install langchain-core langgraph>0.2.27

Note: you may need to restart the kernel to use updated packages.


In [9]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [10]:
config = {"configurable": {"thread_id": "abc123"}}

In [11]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [12]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [13]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [14]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

Note: you may need to restart the kernel to use updated packages.


Getting the Weather Forecast using API call

In [ ]:
import requests

def get_weather(city_name, api_key):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    
    params = {
        'q': city_name,
        'appid': api_key,
        'units': 'metric' 
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        weather = {
            'city': data['name'],
            'temperature': data['main']['temp'],
            'weather': data['weather'][0]['description'],
            'humidity': data['main']['humidity'],
            'wind_speed': data['wind']['speed']
        }
        return weather
    else:
        return {'error': f"Failed to get weather data: {response.status_code}"}

api_key = "OPEN_WEATHER_API_KEY"
city = "Kolkata"

weather_info = get_weather(city, api_key)

print(weather_info)


{'city': 'Kolkata', 'temperature': 30.97, 'weather': 'haze', 'humidity': 74, 'wind_speed': 4.12}


In [24]:
weather_info = get_weather("Delhi",api_key)
print(weather_info)

{'city': 'Delhi', 'temperature': 28.05, 'weather': 'haze', 'humidity': 69, 'wind_speed': 3.09}


In [25]:
weather_info = get_weather("Guwahati",api_key)
print(weather_info)

{'city': 'Guwahati', 'temperature': 31.95, 'weather': 'haze', 'humidity': 74, 'wind_speed': 5.14}


Getting trending information for various city using Tavily Search

In [16]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is trending in Tokyo ?")
print(search_results)
tools = [search]

[{'title': 'Trending in Tokyo - YouTube', 'url': 'https://www.youtube.com/playlist?list=PLKr_Y768pGvDA3wx-nMhCapTkD6SheNw4', 'content': 'Trending in Tokyo · Japanese Love Shrines and Fortunes | Hikawa Shrine in Japan Kawagoe 川越 ❤️ · Sanrio PuroLand Little Twin Stars Christmas | Vlogmas 2019.', 'score': 0.66959417}, {'title': 'New & Trending | The Official Tokyo Travel Guide, GO TOKYO', 'url': 'https://www.gotokyo.org/en/new-and-now/new-and-trending/index.html', 'content': "Search results: 0\n\n\n\nSearch\n\n\n\nMain content starts here.\n\nNew & Trending\n\n\n\nExplore Tokyo Using the Metaverse and AR\n\nCraftsmanship in Tokyo: Japanese Craft Workshops Led by Skilled Artisans\n\nPerfectly Placed Cafes for Cherry Blossom Fans\n\nInspiring Female Chef-owned Restaurants in Tokyo\n\nIndulge in a Luxurious Moment at Tokyoâ\x80\x99s Chocolate Shops this Valentineâ\x80\x99s\n\nExplore Koenji: Tokyo's Live Music Scene\n\nEnjoy Tokyoâ\x80\x99s Winter Charm at an Ice Skating Rink", 'score': 0.66

In [17]:
search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is trending in Kolkata ?")
print(search_results)
tools = [search]

[{'title': 'Kolkata Trends (@kolkatatrends) • Instagram photos and videos', 'url': 'https://www.instagram.com/kolkatatrends/?hl=en', 'content': "Kolkata Trends is your one-stop fashion destination for trendy men's and women's apparel, stylish accessories, and more 🤩 Whether you're looking to elevate", 'score': 0.7476348}, {'title': 'A weird trend : r/kolkata - Reddit', 'url': 'https://www.reddit.com/r/kolkata/comments/1i1syhs/a_weird_trend/', 'content': '403K Members 68 Online\n•4 mo. ago\nmyths_dark1\nA weird trend\nJust for Fun | নিছক মজা 🤡\nI Kolkata spotted a weird trend in most of the shops, People were wearing jackets and sweaters also aircon or fan running in full fledged. I was like whaaatt!! Why dont you turn of Aircon amd switch to tshirt or remove your jackets atleast 😅. Is wearing winter wear a trend or something?\nRead more \nShare  Share \nSort by: \nBest\nOpen comment sort options\n\nBest \nTop \nNew \nControversial \nOld \nQ&A [...] What were they thinking? r/kolkata•8 

In [23]:
search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is trending in Delhi ?")
print(search_results)
tools = [search]

[{'title': 'Delhi News - The Indian Express', 'url': 'https://indianexpress.com/section/cities/delhi/', 'content': 'Subscribe\nSign In\nTrending\n\nIndia Pakistan News\nMini Crossword\nExpress Shorts\n🎙️ Podcast\n📹 Videos\nHealth & Wellness\n\nAdvertisement\n\nNews\nCities\nDelhi\n\nDELHI NEWS\n‘This is the reality of the BJP’: AAP MP Sanjay Singh hits out at MP minister Kunwar Vijay Shah\n\n\n\nNo arrest yet after Gurgaon shopkeeper shot dead for refusing to deliver samosa\n\n\n\n‘Insult to daughters of the nation’: Amid row over MP minister’s jibe, NCW chief Vijaya Rahatkar bats for Colonel Sofiya Qureshi [...] Explained News\nPolitical Pulse\nLatest Opinion\nMumbai News\nDelhi News\nPune News\nBangalore News\nBollywood News\nHealth News\nIndia News\nSports News\nLifestyle News\nLatest News\nCricket\nTech Reviews\nGadgets\nMobile & Tabs\nFood & Wine\nElections\nFitness\n\nTrending News [...] e-paper\nPremium\nIndia\nElections 2024\nBollywood\n\n\n\n\nOpinion\nPolitical Pulse\nExplain